# AI Second Project - Food_DataSet - Training_Models :)

## Episode1 - Read Data And Doing Some Pre_Processing

### Arc1 - Split Data Training

In [1]:
# import libraries we want
# pandas and numpy to work with dataFrame
import pandas as pd
import numpy as np
# tensorflow for make model
import tensorflow as tf
from tensorflow . keras . preprocessing.image import ImageDataGenerator
from tensorflow . keras . applications import MobileNetV2, ResNet50
from tensorflow . keras . layers import Dense, Dropout
from tensorflow . keras . models import Model
from sklearn . model_selection import train_test_split
# PIL library to use some process on image easy
from PIL import Image
import io
# use in the last part to have some correcting
import matplotlib.pyplot as plt

In [2]:
def preprocess_image(image_bytes):
    # open the image from raw bytes
    image = Image.open(io.BytesIO(eval(image_bytes)))
    # resize image to a fixed size
    image = image.resize((224, 224))
    # convert image to numpy array
    image_array = np.array(image)
    return image_array

In [3]:
# create generators from a dataframe
def create_generators(df):
    # use previous function on main DataFrame
    df['Photo'] = df['Photo'] . apply(preprocess_image)
    df = df[df['Photo'].apply(lambda x: x.shape == (224, 224, 3))]
    # split data into 2 part_ train and validation part 
    X_train, X_val, y_train, y_val = train_test_split(np . stack(df['Photo'] . values), df['Label'] . values, test_size = 0.2, random_state = 42)
    # make generators - we only use generator on train data and we don't change anything in validation data
    train_datagen = ImageDataGenerator(
    rescale = 1./255,  # Normalize pixel values to [0,1]
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
    )
    val_datagen = ImageDataGenerator(rescale = 1./255)
    # use above augmentation for data
    train_generator = train_datagen . flow(X_train, y_train, batch_size = 32)
    val_generator = val_datagen . flow(X_val, y_val, batch_size = 32)
    return train_generator, val_generator

In [4]:
# check df1
df1 = pd . read_csv('high.csv')  # High Accuracy

In [5]:
train_gen1, val_gen1 = create_generators(df1)

In [4]:
df2 = pd . read_csv('mid.csv')

In [8]:
train_gen2, val_gen2 = create_generators(df2)

In [4]:
df3 = pd . read_csv('low.csv')

In [5]:
train_gen3, val_gen3 = create_generators(df3)

### Arc2 - All Data Maker

In [2]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to create training and validation generators from a directory
def create_generators(image_dir, img_size=(224, 224), batch_size=32):
    # ImageDataGenerator with data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,                  # Normalize pixel values to [0, 1]
        rotation_range=20,               # Randomly rotate images
        zoom_range=0.2,                  # Randomly zoom images
        width_shift_range=0.2,           # Randomly shift images horizontally
        height_shift_range=0.2,          # Randomly shift images vertically
        horizontal_flip=True,            # Randomly flip images horizontally
        fill_mode='nearest',             # Fill pixels when shifts happen
        validation_split=0.2             # Split data into training and validation
    )
    
    # ImageDataGenerator for validation (only rescaling, no augmentation)
    val_datagen = ImageDataGenerator(
        rescale=1./255,                  # Normalize pixel values to [0, 1]
        validation_split=0.2             # Same split as training
    )
    
    # Training generator
    train_generator = train_datagen.flow_from_directory(
        image_dir,
        target_size=img_size,            # Resize images to the specified size
        batch_size=batch_size,           # Number of images per batch
        class_mode='categorical',        # Use 'categorical' since it's a multi-class classification problem
        subset='training',               # Use the training subset
        shuffle=True                     # Shuffle images for training
    )
    
    # Validation generator
    val_generator = val_datagen.flow_from_directory(
        image_dir,
        target_size=img_size,            # Resize images to the specified size
        batch_size=batch_size,           # Number of images per batch
        class_mode='categorical',        # Use 'categorical' for multi-class classification
        subset='validation',             # Use the validation subset
        shuffle=False                    # No shuffle for validation
    )
    
    # Ensure that both training and validation have 101 classes
    assert len(train_generator.class_indices) == 101, "Training set doesn't contain 101 classes."
    assert len(val_generator.class_indices) == 101, "Validation set doesn't contain 101 classes."
    
    return train_generator, val_generator

# Example usage
image_dir = "images"
train_gen, val_gen = create_generators(image_dir)

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


## Episode2 - Train The 3 File 3 Times to Model Converge to Best Result

### Part1 Training - No Trainable Layer

In [10]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(101, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model . trainable = True
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [66]:
# make model
model = create_model()
# give summary
model . summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_13[0][0]']               
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [67]:
model . compile(optimizer = tf.keras.optimizers.Adam(1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [68]:
model . fit(train_gen, validation_data = val_gen, epochs = 5)
# save the weights of model in .h5 file
model . save_weights('model1.h5')

Epoch 1/5
2525/2525 [==============================] - 3430s 1s/step - loss: 2.8211 - accuracy: 0.3179 - val_loss: 2.0770 - val_accuracy: 0.4769
Epoch 2/5
2525/2525 [==============================] - 3546s 1s/step - loss: 2.5067 - accuracy: 0.3824 - val_loss: 2.0084 - val_accuracy: 0.4962
Epoch 3/5
2525/2525 [==============================] - 2065s 817ms/step - loss: 2.4462 - accuracy: 0.3963 - val_loss: 1.9977 - val_accuracy: 0.4953
Epoch 4/5
2525/2525 [==============================] - 1938s 768ms/step - loss: 2.4069 - accuracy: 0.4078 - val_loss: 1.9590 - val_accuracy: 0.5099
Epoch 5/5
2525/2525 [==============================] - 1936s 767ms/step - loss: 2.3909 - accuracy: 0.4130 - val_loss: 1.9678 - val_accuracy: 0.5084


### Part2 - 100 First Layer Is Non_Trainable

In [24]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(101, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model . trainable = True
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [25]:
# make model
model = create_model()
# load model weights that we make in previous part
model . load_weights('model1.h5')
# all layers of base model is trainable
model . trainable = True
# but we freeze 100 first layer
for layer in model . layers[:120]:
    layer . trainable = False
model . summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [27]:
model . compile(optimizer = tf.keras.optimizers.Adam(1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [28]:
# compile model and training
model . fit(train_gen, validation_data = val_gen, epochs = 5)
# save model
model . save_weights('model2.h5')

Epoch 1/5
2525/2525 [==============================] - 3408s 1s/step - loss: 2.1716 - accuracy: 0.4632 - val_loss: 1.8186 - val_accuracy: 0.5488
Epoch 2/5
2525/2525 [==============================] - 1824s 722ms/step - loss: 1.8205 - accuracy: 0.5361 - val_loss: 1.8497 - val_accuracy: 0.5545
Epoch 3/5
2525/2525 [==============================] - 1703s 674ms/step - loss: 1.6298 - accuracy: 0.5824 - val_loss: 1.5250 - val_accuracy: 0.6149
Epoch 4/5
2525/2525 [==============================] - 1697s 672ms/step - loss: 1.4956 - accuracy: 0.6124 - val_loss: 1.4923 - val_accuracy: 0.6286
Epoch 5/5
2525/2525 [==============================] - 1687s 668ms/step - loss: 1.3906 - accuracy: 0.6351 - val_loss: 1.4854 - val_accuracy: 0.6380


## Episode3 - Train The High.csv And Mid.csv File 3 Times to Model Converge to Best Result

### Part1 Training - No Trainable Layer

In [9]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(101, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model . trainable = True
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [10]:
# make model - in first part we don't need add any trainable layer
model = create_model()
model . load_weights('model5.h5')
model . summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [12]:
# train model
model . compile(optimizer = tf . keras.optimizers . Adam(1e-3), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model . fit(train_gen2, validation_data = val_gen2, epochs = 5)
# save model
model . save_weights('model6.h5')

Epoch 1/5
853/853 [==============================] - 565s 659ms/step - loss: 1.9267 - accuracy: 0.5279 - val_loss: 1.0257 - val_accuracy: 0.7061
Epoch 2/5
853/853 [==============================] - 585s 686ms/step - loss: 1.1592 - accuracy: 0.6736 - val_loss: 0.9474 - val_accuracy: 0.7297
Epoch 3/5
853/853 [==============================] - 590s 691ms/step - loss: 1.0472 - accuracy: 0.7025 - val_loss: 0.9339 - val_accuracy: 0.7305
Epoch 4/5
853/853 [==============================] - 584s 684ms/step - loss: 0.9761 - accuracy: 0.7177 - val_loss: 0.9173 - val_accuracy: 0.7368
Epoch 5/5
853/853 [==============================] - 571s 669ms/step - loss: 0.9306 - accuracy: 0.7298 - val_loss: 0.9178 - val_accuracy: 0.7411


### Part2 - 100 First Layer Is Non_Trainable

In [13]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(101, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model . trainable = True
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [14]:
# make model
model = create_model()
# load model weights that we make in previous part
model . load_weights('model6.h5')
# all layers of base model is trainable
model . trainable = True
# but we freeze 100 first layer
for layer in model . layers[:100]:
    layer . trainable = False
model . summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [15]:
model . compile(optimizer = tf.keras.optimizers.Adam(1e-4), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [16]:
# compile model and train it
model . fit(train_gen2, validation_data = val_gen2, epochs = 5)
# save model
model . save_weights('model7.h5')

Epoch 1/5
853/853 [==============================] - 765s 890ms/step - loss: 0.9376 - accuracy: 0.7298 - val_loss: 0.8972 - val_accuracy: 0.7550
Epoch 2/5
853/853 [==============================] - 714s 837ms/step - loss: 0.7894 - accuracy: 0.7718 - val_loss: 0.8119 - val_accuracy: 0.7782
Epoch 3/5
853/853 [==============================] - 708s 829ms/step - loss: 0.7068 - accuracy: 0.7940 - val_loss: 0.8750 - val_accuracy: 0.7695
Epoch 4/5
853/853 [==============================] - 700s 821ms/step - loss: 0.6467 - accuracy: 0.8096 - val_loss: 0.8901 - val_accuracy: 0.7651
Epoch 5/5
853/853 [==============================] - 735s 862ms/step - loss: 0.5852 - accuracy: 0.8269 - val_loss: 0.8412 - val_accuracy: 0.7826


## Episode4 - Train The Low.csv File 3 Times to Model Converge to Best Result

### Part1 Training - No Trainable Layer

In [6]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(101, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model . trainable = True
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [7]:
# make model and load weights
model = create_model()
model.load_weights('model7.h5')

In [8]:
model . compile(optimizer = tf . keras.optimizers . Adam(1e-3), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model . fit(train_gen3, validation_data = val_gen3, epochs = 5)
model . save_weights('model8.h5')

Epoch 1/5
753/753 [==============================] - 492s 650ms/step - loss: 1.8866 - accuracy: 0.5696 - val_loss: 0.7800 - val_accuracy: 0.7849
Epoch 2/5
753/753 [==============================] - 480s 637ms/step - loss: 0.9223 - accuracy: 0.7460 - val_loss: 0.7039 - val_accuracy: 0.8010
Epoch 3/5
753/753 [==============================] - 501s 666ms/step - loss: 0.7940 - accuracy: 0.7782 - val_loss: 0.6693 - val_accuracy: 0.8081
Epoch 4/5
753/753 [==============================] - 483s 641ms/step - loss: 0.7373 - accuracy: 0.7914 - val_loss: 0.6348 - val_accuracy: 0.8213
Epoch 5/5
753/753 [==============================] - 475s 631ms/step - loss: 0.6800 - accuracy: 0.8064 - val_loss: 0.6355 - val_accuracy: 0.8224


### Part2 - 100 First Layer Is Non_Trainable

In [14]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(101, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model . trainable = True
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [15]:
# make model
model = create_model()
# load model weights that we make in previous part
model . load_weights('model8.h5')
# all layers of base model is trainable
model . trainable = True
# but we freeze 100 first layer
for layer in model . layers[:100]:
    layer . trainable = False
model . summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [16]:
# compile model
model . compile(optimizer = tf.keras.optimizers.Adam(1e-4), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [17]:
# compile model and train
model . fit(train_gen3, validation_data = val_gen3, epochs = 5)
# save model
model . save_weights('model9.h5')

Epoch 1/5
753/753 [==============================] - 654s 861ms/step - loss: 0.6966 - accuracy: 0.8018 - val_loss: 0.5933 - val_accuracy: 0.8384
Epoch 2/5
753/753 [==============================] - 634s 842ms/step - loss: 0.5858 - accuracy: 0.8328 - val_loss: 0.6122 - val_accuracy: 0.8365
Epoch 3/5
753/753 [==============================] - 635s 843ms/step - loss: 0.5024 - accuracy: 0.8567 - val_loss: 0.5681 - val_accuracy: 0.8485
Epoch 4/5
753/753 [==============================] - 639s 848ms/step - loss: 0.4544 - accuracy: 0.8699 - val_loss: 0.5339 - val_accuracy: 0.8625
Epoch 5/5
753/753 [==============================] - 616s 817ms/step - loss: 0.4005 - accuracy: 0.8829 - val_loss: 0.5569 - val_accuracy: 0.8561


## Episode5 - Train Model On ConCat ALL 3 DataFrames 

### Part1 - 100 FIrst Layer is Non_Trainable

In [6]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(101, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model . trainable = True
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [7]:
model = create_model()
model . load_weights('model9.h5')
model . summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [8]:
# compile model
model . compile(optimizer = tf.keras.optimizers.Adam(1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [9]:
# compile model and train
model . fit(train_gen, validation_data = val_gen, epochs = 5)
# save model
model . save_weights('model_final_1.h5')

Epoch 1/5
2525/2525 [==============================] - 3392s 1s/step - loss: 1.9560 - accuracy: 0.5467 - val_loss: 1.3511 - val_accuracy: 0.6559
Epoch 2/5
2525/2525 [==============================] - 1488s 589ms/step - loss: 1.2929 - accuracy: 0.6698 - val_loss: 1.2794 - val_accuracy: 0.6741
Epoch 3/5
2525/2525 [==============================] - 1473s 583ms/step - loss: 1.1930 - accuracy: 0.6907 - val_loss: 1.2736 - val_accuracy: 0.6743
Epoch 4/5
2525/2525 [==============================] - 1462s 579ms/step - loss: 1.1451 - accuracy: 0.7012 - val_loss: 1.2496 - val_accuracy: 0.6827
Epoch 5/5
2525/2525 [==============================] - 1469s 582ms/step - loss: 1.1059 - accuracy: 0.7108 - val_loss: 1.2444 - val_accuracy: 0.6881


### Part2 - All Layers Are Non_Trainable And Dropout is 0.2 lower

In [30]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(101, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model . trainable = True
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [31]:
model = create_model()
model . load_weights('model_final_1.h5')
model . summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [32]:
# initialize values of warmup_learning_rate
warmup_steps = 5
initial_lr = 1e-3
target_lr = 1e-5
# compile model
model . compile(optimizer = tf.keras.optimizers.Adam(initial_lr), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [33]:
model . fit(train_gen, validation_data = val_gen, epochs = 5)
# save weights
model . save_weights('model_final_2.h5')

Epoch 1/5
2525/2525 [==============================] - 2234s 884ms/step - loss: 1.0860 - accuracy: 0.7175 - val_loss: 1.2962 - val_accuracy: 0.6780
Epoch 2/5
2525/2525 [==============================] - 1642s 650ms/step - loss: 1.0561 - accuracy: 0.7225 - val_loss: 1.2593 - val_accuracy: 0.6887
Epoch 3/5
2525/2525 [==============================] - 1678s 665ms/step - loss: 1.0420 - accuracy: 0.7266 - val_loss: 1.2847 - val_accuracy: 0.6861
Epoch 4/5
2525/2525 [==============================] - 1618s 641ms/step - loss: 1.0162 - accuracy: 0.7299 - val_loss: 1.2879 - val_accuracy: 0.6867
Epoch 5/5
2525/2525 [==============================] - 1549s 614ms/step - loss: 1.0035 - accuracy: 0.7344 - val_loss: 1.2983 - val_accuracy: 0.6862
